# LP: Záměrně neproveditelná úloha v PuLP

Cíl: vytvořit triviální LP s konfliktními omezeními, spustit CBC přes PuLP, a ukázat, že solver vrátí stav *Infeasible*.

**Model**
- Proměnná: $x \in \mathbb{R}$
- Minimalizace: $ \min x $
- Omezení: $x \ge 1$ a $x \le 0{,}5$  ⟶ konflikt ⟶ neproveditelné

Postup:
1) Instalace a import PuLP.  
2) Sestavení modelu.  
3) Řešení s výpisem logu solveru.  
4) Kontrola stavu, hodnot proměnných a účelové funkce.  
5) Uložení modelu do `infeasible.lp` pro inspekci.


In [ ]:
# 1) Instalace a import PuLP (robustně)
try:
    import pulp
except ImportError:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pulp"])
    import pulp

print("PuLP verze:", pulp.__version__)
print("Dostupní řešitelé:", pulp.listSolvers(onlyAvailable=True))


PuLP verze: 3.3.0
Dostupní řešitelé: ['PULP_CBC_CMD', 'HiGHS']


## Sestavení neproveditelné LP úlohy

Úmyslně nastavíme dvě vzájemně se vylučující podmínky na jednu proměnnou $x$:
- $x \ge 1$
- $x \le 0{,}5$

Očekávaný výsledek: solver nahlásí stav *Infeasible*.


In [ ]:
# 2) Formulace modelu v PuLP
from pulp import LpProblem, LpMinimize, LpVariable, LpStatus, value, PULP_CBC_CMD

prob = LpProblem("infeasible_demo", LpMinimize)

# Proměnná x je reálná, bez omezení znakem; omezení přidáme explicitně
x = LpVariable("x", lowBound=None, upBound=None, cat="Continuous")

# Cíl: min x
prob += x, "objective"

# Konfliktní omezení
prob += (x >= 1),  "c1_lower_bound"
prob += (x <= 0.5),"c2_upper_bound"

# Volitelně: uložit LP do souboru pro kontrolu
prob.writeLP("infeasible.lp")
print("Model uložen do: infeasible.lp")


Model uložen do: infeasible.lp


In [ ]:
# 3) Řešení s CBC, zapnuté logování
solver = PULP_CBC_CMD(msg=True)  # msg=True = zobrazit log CBC
status_code = prob.solve(solver)

print("\n--- Souhrn stavu ---")
print("Status kód:", status_code)
print("Status text:", LpStatus[prob.status])

# 4) Kontrola hodnot proměnných a účelové funkce
def safe_var(v):
    return None if v.varValue is None else float(v.varValue)

obj_val = None
try:
    obj_val = value(prob.objective)
except Exception:
    obj_val = None

print("x =", safe_var(x))
print("Hodnota účelové funkce =", obj_val)



--- Souhrn stavu ---
Status kód: -1
Status text: Infeasible
x = 1.0
Hodnota účelové funkce = 1.0


## Co sledovat ve výstupu

V logu CBC uvidíte hlášení indikující neproveditelnost (např. závěr o *infeasible* modelu).  
V proměnných i cíli budou hodnoty `None`, protože řešení neexistuje.

Poznámky:
- Soubor `infeasible.lp` můžete otevřít v libovolném LP editoru nebo znovu řešit externě.
- Pro systematickou diagnostiku se běžně používají *conflict refiner* nebo *IIS* (minimální neslučitelná množina); CBC jako samostatný balík takový výstup standardně neposkytuje, ale lze využít jiné solvery nebo manuální analýzu konfliktů.
